<a href="https://colab.research.google.com/github/doublesoftie/ficbook_project/blob/main/Ebeyshie_progeru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests

In [ ]:
!pip install requests

In [ ]:
!pip install fake_useragent

In [ ]:
from fake_useragent import UserAgent

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
from types import NoneType

In [ ]:
import csv

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from types import NoneType
# здесь мы сначала переходим на страницу фанфика и только потом начинаем собирать данные
def get_page(p):

    # изготовили ссылку
    url = f'https://ficbook.net/find-fanfics-846555?fandom_filter=any&fandom_group_id=1&pages_range=1&pages_min=&pages_max=&transl=1&likes_min=&likes_max=&rewards_min=&date_create_min=2024-04-01&date_create_max=2024-05-01&date_update_min=2024-04-01&date_update_max=2024-05-01&title=&sort=5&rnd=1914647106&find=Найти%21#result'
    response = requests.get(url, headers={'User-Agent': UserAgent().chrome})

    # построили дерево
    tree = BeautifulSoup(response.content, 'html.parser')

    # собираем ссылки на фанфики на странице
    fanfics = tree.find_all('a', {'class': 'visit-link'})

    infa = [ ]

    for fanfic in fanfics:

        # берем ссылку на фанфик и переходим на его страницу, сохраняем дерево фанфика
        link = fanfic.get('href')
        url = f'https://ficbook.net{link}'
        response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
        tree_fanfic = BeautifulSoup(response.content, 'html.parser')

        # добавляем название фанфика и ссылку на него
        discription ={'название': tree_fanfic.h1.text,
                      'ссылка': f'https://ficbook.net{link}'}

        # находим 1ю группу характеристик, принадлежащих классу 'badge-text'
        features1 = tree_fanfic.find_all('span', {'class': 'badge-text'})

        # в зависимости от перевода/оригинала фанфик имеет разную последовательность характеристик
        if features1[1].text == 'Перевод':

          # так как может не быть количество лайклв как здесь https://ficbook.net/readfic/1009396 ставим условие
          if len(features1)  >= 5:
            discription.update({'тип отношений': features1[0].text,
                                'возрастной рейтинг': features1[2].text,
                                'статус завершенности': features1[3].text,
                                'количество лайков': features1[4].text})
          else:
            print(url)
        else:
          if len(features1)  >= 4:  # аналогично может не быть количества лайков
            discription.update({'тип отношений': features1[0].text,
                                'возрастной рейтинг': features1[1].text,
                                'статус завершенности': features1[2].text,
                                'количество лайков': features1[3].text})
          else:
            print(url)

        # находим вторую группу характеристик, принадлежащих классу 'mb-5'
        features2 = tree_fanfic.find_all('div', {'class': 'mb-5'})
        for el in features2:
            if el.strong.text == 'Описание:':
                discription['описание'] = el.div.text
            if el.strong.text == 'Размер:':
                discription['размер'] = el.div.text.strip()
            if el.strong.text == 'Метки:':
              discription['количество меток'] = len(el.find_all('a', {'class': 'tag'}))

        # находим количество отзывов по фанфику из 3 группы характеристик
        features3 = tree_fanfic.find_all('a', {'class': 'btn btn-primary btn-with-description'})
        for el in features3:
          if el.find('span', {'class': 'description'}).text == 'Отзывы':
              discription['количество отзывов'] = el.find('span', {'class': 'main-info'}).text.strip()

        # находим ссылку на фандом и находим количество страниц, написанных по нему
        link_fandom = tree_fanfic.find('div', {'class': 'mb-10'}).find('a').get('href')
        url = f'https://ficbook.net{link_fandom}'
        response = requests.get(url, headers={'User-Agent': UserAgent().chrome})
        tree_fandom = BeautifulSoup(response.content, 'html.parser')
        if tree_fandom.h1.text.split() == ['Ориджиналы']:
            discription['количество страниц по фандому'] = '0'
        else:
            if type(tree_fandom.find('input', {'class': 'jsPageJumper'})) == NoneType: # если страница по фандому всего одна, не будет данных о количестве страниц
                discription['количество страниц по фандому'] = 1
                print(url)
            else:
                discription['количество страниц по фандому'] = tree_fandom.find('input', {'class': 'jsPageJumper'}).get('max')

        # добавляем всю собранную информацию о фанфике
        infa.append(discription)

    return infa

In [ ]:
with open('fickbook.csv', 'a') as output_file:
  keys = ['название', 'ссылка', 'тип отношений', 'возрастной рейтинг', 'статус завершенности', 'количество лайков', 'размер', 'количество меток', 'описание', 'количество отзывов', 'количество страниц по фандому']
  dict_writer = csv.DictWriter(output_file, keys)
  dict_writer.writeheader()

In [ ]:
for p in tqdm(range(1)):
  with open('fickbook.csv', 'a') as output_file:
      dict_writer = csv.DictWriter(output_file, keys)
      dict_writer.writerows(get_page(p))

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
import pandas as pd

df = pd.read_csv('fickbook.csv')
print(df.shape)
df

(23, 11)


,название,ссылка,тип отношений,возрастной рейтинг,статус завершенности,количество лайков,размер,количество меток,описание,количество отзывов,количество страниц по фандому
0,Примеряй свои тени (НА РЕДАКТУРЕ),https://ficbook.net/readfic/12092465?source=pr...,Гет,NC-17,Завершён,116,"364 страницы, 30 частей",34,К чему может привести неожиданное интернет-зна...,434.0,0
1,Однажды маски спадают,https://ficbook.net/readfic/018ccb75-96a3-7728...,Слэш,PG-13,Завершён,22,"6 страниц, 1 часть",7,Очередной стрим обернулся для Дипинса самым не...,6.0,497
2,On the edge of eternity,https://ficbook.net/readfic/13142693?source=pr...,Слэш,R,Завершён,102,"10 страниц, 1 часть",9,"- Папуля же поправится, он не умрет? - продолж...",12.0,1000
3,змея в сердце,https://ficbook.net/readfic/7513378,Фемслэш,PG-13,Завершён,42,"1 страница, 1 часть",4,"У Шерил, как оказалось, не просто сердце. У не...",NaN,385
4,"Тебе больше не нужны очки, придурок",https://ficbook.net/readfic/6069259,Слэш,PG-13,Завершён,551,"4 страницы, 1 часть",8,"соулмейт!AU, в котором,если один из соулмейтов...",7.0,431
5,Красная роза!,https://ficbook.net/readfic/647278,Джен,G,Завершён,8,"1 страница, 1 часть",4,Он напишет ей письмо...,4.0,190
6,Заброшенный Склад,https://ficbook.net/readfic/10919160,Джен,G,Завершён,0,"1 страница, 1 часть",3,Стул,0.0,0
7,Одиночество - не выход...,https://ficbook.net/readfic/1291327,Джен,G,Завершён,4,"1 страница, 1 часть",3,Ушёл в себя. Скоро вернусь.,0.0,0
8,Любовь всей жизни,https://ficbook.net/readfic/8136914,Слэш,PG-13,Заморожен,10,"9 страниц, 4 части",4,"Пак Чимин влюблен в своего нового друга, котор...",1.0,1000
9,this guy who slowly destroys orientation,https://ficbook.net/readfic/9017122,Слэш,R,Заморожен,9,"7 страниц, 3 части",5,Тайлер делает каминг-аут другу/учителю. И у Дж...,0.0,282


In [ ]:
df.duplicated().sum()

1

In [ ]:
df.iloc[12][1]

'https://ficbook.net/readfic/018cf997-595c-73af-80d3-dc50c8d9c47c'